In [1]:
import baml_py
import jupyter_black
from unittest.mock import Mock
from pprint import pprint

import explorations.d250428_portable_baml_parsing.f02_portable_baml_parser_types as types
from baml_agents._agent_tools._json_schema_to_baml_source._json_to_model import (
    JsonSchemaToBamlModelConverter,
)
from baml_agents._agent_tools._json_schema_to_baml_source._model_to_baml_source import (
    BamlModelToBamlSourceConverter,
)

jupyter_black.load()

return_type = types.Person.__name__
print(f"{return_type=}")

return_type='Person'


In [2]:
json_schema = types.Person.model_json_schema()
pprint(json_schema)

{'properties': {'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
                        'default': None,
                        'description': 'Age in years which must be equal to or '
                                       'greater than zero.',
                        'title': 'Age'},
                'first': {'description': "The person's first name.",
                          'title': 'First',
                          'type': 'string'},
                'last': {'description': "The person's last name.",
                         'title': 'Last',
                         'type': 'string'}},
 'required': ['first', 'last'],
 'title': 'Person',
 'type': 'object'}


In [3]:
baml_models = JsonSchemaToBamlModelConverter(json_schema, return_type).convert()
baml_source = BamlModelToBamlSourceConverter(baml_models).generate()

full_baml_source = f"""\
{baml_source}
function Virtual() -> {return_type} {{
  client "openai/gpt-4o"
  prompt #""#
}}"""
print(full_baml_source)

class Person {
  first string @description(#"The person's first name."#)
  last string @description(#"The person's last name."#)
  age int? @description(#"Age in years which must be equal to or greater than zero."#)
}

function Virtual() -> Person {
  client "openai/gpt-4o"
  prompt #""#
}


In [4]:
baml_runtime = baml_py.BamlRuntime.from_files(
    root_path="baml_src", files={"virtual.baml": full_baml_source}, env_vars={}
)
baml_ctx = baml_py.BamlCtxManager(baml_runtime).get()

In [5]:
llm_response = """\
Here is your answer
{
  first: "John",
  "last": Doe Johnson,
  age: "25"
}
Let me know if you need anything else
"""

person = baml_runtime.parse_llm_response(
    "Virtual",
    llm_response,
    types,  # enum_module
    types,  # cls_module
    types,  # partial_cls_module
    False,  # allow_partials
    baml_ctx.deep_clone(),  # ctx
    None,  # tb
    None,  # cr
)
print(f"{person.first=}\n{person.last=}\n{person.age=}")

person.first='John'
person.last='Doe Johnson'
person.age=25
